In [36]:
# Importing necessary modules
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [37]:
!pip install torch-summary 
from torchsummaryX import summary as summaryX
from torchsummary import summary

In [38]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 6,
                              kernel_size = 5, stride=1, padding =0)
        self.conv2 = nn.Conv2d(in_channels = 6, out_channels = 16,
                              kernel_size = 5, stride=1 , padding = 0)
        self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 120,
                              kernel_size = 5, stride=1, padding = 0)
        self.linear1 = nn.Linear(120,84)
        self.linear2 = nn.Linear(84,10)
        self.tanh = nn.Tanh()
        self.avgpool = nn.AvgPool2d(kernel_size = 2, stride = 2)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.tanh(x)
        x = self.avgpool(x)
        x = self.conv2(x)
        x = self.tanh(x)
        x = self.avgpool(x)
        x = self.conv3(x)
        x = self.tanh(x)
        
        x = x.reshape(x.shape[0], -1)
        x = self.linear1(x)
        x = self.tanh(x)
        x = self.linear2(x)
        
        return x

In [39]:
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

In [40]:
model = LeNet()
x = torch.randn(64,1,32,32)
output = model(x)
print(output.shape)

torch.Size([64, 10])


In [41]:
print(model)
summary(model, (1,32,32))
print("output.shape", output.shape)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (linear1): Linear(in_features=120, out_features=84, bias=True)
  (linear2): Linear(in_features=84, out_features=10, bias=True)
  (tanh): Tanh()
  (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
)
Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 6, 28, 28]           156
├─Tanh: 1-2                              [-1, 6, 28, 28]           --
├─AvgPool2d: 1-3                         [-1, 6, 14, 14]           --
├─Conv2d: 1-4                            [-1, 16, 10, 10]          2,416
├─Tanh: 1-5                              [-1, 16, 10, 10]          --
├─AvgPool2d: 1-6                         [-1, 16, 5, 5]            --
├─Conv2d: 1-7                            [-1, 120, 1, 1]           48,120
├─Tanh: 1-8                     

In [42]:
# Loading Mnist

# Hyperparameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
learning_rate = 0.01
num_epochs = 10
batch_size = 64
load_model = True

train_dataset = datasets.MNIST(root = 'dataset/', train =True, 
                               transform = transform, download = True)
test_dataset = datasets.MNIST(root = 'dataset/', train =False, 
                               transform = transform, download = True)

In [43]:
# Dataloader
train_loader = DataLoader(dataset = train_dataset, batch_size=64, shuffle = True)
test_loader = DataLoader(dataset = test_dataset, batch_size=64, shuffle = True)

# dataset_sizes = {'train':len(train_dataset), 'test':len(test_dataset)}

In [44]:
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [46]:
# print("Before backward pass : \n", model[0].weight.grad)
# loss.backward()
# print("After backward pass : \n", model[0].weight.grad)

In [47]:
# from IPython.display import HTML, display

# class ProgessMonitor(object):
#     # Custom IPython progess bar for training
    
#     tmpl = '''
    
#           <p> Loss : {loss:0.4f}  {value}/{length}</p>
#           <progess value = '{value} max={length}', style='width:100%'>{value}</progess>
#     '''
    
    
#     def __init__(self,length):
#         self.length = length
#         self.count = 0
#         self.display = display(self.html(0,0), display_id = True)

#     def html(self,count,loss):
#         return HTML(self.tmpl.format(length=self.length, value = count, loss=loss))
    
#     def update(self, count, loss):
#         self.count += count
#         self.display.update(self.html(self.count, loss))
        
        

In [48]:
# Train Network

for epoch in range(num_epochs):
    for batch_idx,(data,targets) in enumerate(train_loader):
        
        #Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        #forward
        scores = model(data)
        loss = criterion(scores,targets)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        #gradient descent or adam step
        optimizer.step()

RuntimeError: Calculated padded input size per channel: (4 x 4). Kernel size: (5 x 5). Kernel size can't be greater than actual input size